In [103]:
import pandas as pd
import numpy as np

from splinter import Browser
from bs4 import BeautifulSoup

In [92]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [93]:
url = 'https://www.mtggoldfish.com/metagame/pauper#paper'
browser.visit(url)
#html = browser.html
#soup = BeautifulSoup(html, 'html.parser')

In [94]:
card_name = []
num_card = []
card_price = []

html_tournaments = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_tournaments, 'html.parser')


tournaments = soup.find('div', class_ = 'decks-sidebar').find_all('h3')[1:5]

# Store the main_url 
tournament_main_url = 'https://www.mtggoldfish.com'


for tournament in tournaments:
    
    link = tournament.find('a')
    tournament_link = link['href']
    #print(tournament_link)
    
    browser.visit(tournament_main_url + tournament_link)
    
    html_decks = browser.html

    soup = BeautifulSoup(html_decks, 'html.parser')
    
    decks = soup.find('table', class_ = 'table table-condensed table-bordered table-tournament').find_all('tr')[1:5]
    
    #print(decks)
    
    for deck in decks[::2]:
        #print('----')
        league_links = deck.find('a')
        
        
        #print(league_links)
        try:
            deck_link = league_links['href']
        except:
            pass
        
        #print(deck_link)
        
        browser.visit(tournament_main_url + deck_link)
        
        html_deck = browser.html

        soup = BeautifulSoup(html_deck, 'html.parser')

        cards = soup.find('table', class_ = 'deck-view-deck-table').find_all('tr')
         
        names = soup.find_all('td', class_ = 'deck-col-card')
        #print(names)
        
        for name in names:  
            try:
                if (name):
                    card_name.append(name.text.replace('\n', ''))
            except:
                pass
        
        num_cards = soup.find_all('td', class_ = 'deck-col-qty')
        #print(cards)
        
        for numb_card in num_cards:  
            try:
                if (numb_card):
                    num_card.append(numb_card.text.replace('\n', ''))
            except:
                pass
        
        
        #break

In [95]:
#print(decks[1])
#print(card_name)

In [96]:
tournament = pd.DataFrame({'Card Name':card_name, 'Card Number':num_card})
tournament.head()

,Card Name,Card Number
0,Ghitu Lavarunner,4
1,Thermo-Alchemist,4
2,Chain Lightning,4
3,Lava Spike,4
4,Lightning Bolt,4
5,Needle Drop,4
6,Searing Blaze,4
7,Rift Bolt,4
8,Skewer the Critics,4
9,Fireblast,4


In [121]:
unique_cards = tournament['Card Name'].unique()



unique_count = []

for card in unique_cards:    
    unique_count.append(tournament.loc[tournament['Card Name'] == card]['Card Number'].astype('int').sum()/3)
    pass

unique_count

card_db = pd.DataFrame({'Name':unique_cards, 'Copies':unique_count})
card_db

,Name,Copies
0,Ghitu Lavarunner,4.0
1,Thermo-Alchemist,4.0
2,Chain Lightning,4.0
3,Lava Spike,4.0
4,Lightning Bolt,4.0
...,...,...
139,Urban Utopia,1.0
140,Snow-Covered Forest,14.0
141,Snow-Covered Plains,1.0
142,Duress,2.0


In [123]:
card_db.to_csv('raw_count.csv')